<a href="https://colab.research.google.com/github/reachosen/ProactiveUtilizationReviewMLR/blob/main/ProactiveUtilizationReviewMLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Cell 1: Define test cases
test_cases = {
    "Acute Care": [
        {
            "id": "TP1",
            "domain": "Acute Care",
            "description": "Approval of Necessary CT Scan",
            "scenario": "7-year-old male with abdominal pain, vomiting, fever (101.5°F) for 24 hours, right lower quadrant tenderness",
            "service": "Abdominal CT scan",
            "input": {"pain_duration_hours": 24, "fever": 101.5, "tenderness": True, "cost": 1200},
            "expected_decision": "Approved",
            "expected_outcome": "True Positive",
            "reason": "Meets AAP criteria for suspected appendicitis: pain >12h, fever >100.4°F, tenderness"
        },
        {
            "id": "TN1",
            "domain": "Acute Care",
            "description": "Denial of Unnecessary Chest X-Ray",
            "scenario": "4-year-old female with cough, mild fever (100.2°F), runny nose for 2 days; no distress, O2 sat 98%",
            "service": "Chest X-Ray",
            "input": {"fever": 100.2, "duration_days": 2, "o2_sat": 98, "distress": False, "cost": 150},
            "expected_decision": "Denied",
            "expected_outcome": "True Negative",
            "reason": "Does not meet criteria: no distress, O2 sat >95%, fever <101°F, likely viral illness"
        },
        {
            "id": "FP1",
            "domain": "Acute Care",
            "description": "Incorrect Approval of Unnecessary CBC",
            "scenario": "5-year-old male with fever (100.6°F) and fatigue for 12 hours; alert, drinking fluids",
            "service": "CBC",
            "input": {"fever": 100.6, "duration_hours": 12, "systemic_signs": False, "cost": 30},
            "expected_decision": "Approved",
            "expected_outcome": "False Positive",
            "reason": "Should be denied: fever <101°F, <48h, no systemic signs; approved due to error"
        },
        {
            "id": "FN1",
            "domain": "Acute Care",
            "description": "Incorrect Denial of Necessary Admission",
            "scenario": "3-year-old female with asthma exacerbation; O2 sat 92%, wheezing, poor response to nebulizer",
            "service": "Inpatient Admission",
            "input": {"o2_sat": 92, "distress": True, "response_to_treatment": "poor", "cost": 15000},
            "expected_decision": "Denied",
            "expected_outcome": "False Negative",
            "reason": "Should be approved: O2 sat <95%, distress, poor response; denied due to data error"
        },
        {
            "id": "TP2",
            "domain": "Acute Care",
            "description": "Approval of Necessary Admission",
            "scenario": "2-year-old male with severe dehydration from gastroenteritis; vomiting/diarrhea 48 hours, lethargy, no urine 12 hours",
            "service": "Inpatient Admission",
            "input": {"duration_hours": 48, "lethargy": True, "urine_output_hours": 12, "cost": 15000},
            "expected_decision": "Approved",
            "expected_outcome": "True Positive",
            "reason": "Meets criteria: severe dehydration with no urine >8h, lethargy"
        },
        {
            "id": "TN2",
            "domain": "Acute Care",
            "description": "Denial of Unnecessary Admission",
            "scenario": "6-year-old female with fever (100.8°F) and sore throat for 24 hours; alert, drinking fluids",
            "service": "Inpatient Admission",
            "input": {"fever": 100.8, "duration_hours": 24, "systemic_signs": False, "cost": 5000},
            "expected_decision": "Denied",
            "expected_outcome": "True Negative",
            "reason": "Does not meet criteria: fever <102°F, no systemic signs, manageable outpatient"
        },
        {
            "id": "FP2",
            "domain": "Acute Care",
            "description": "Incorrect Approval of Unnecessary Head CT",
            "scenario": "8-year-old male with minor head injury; no LOC, no vomiting, normal neuro exam",
            "service": "Head CT scan",
            "input": {"loc": False, "vomiting": False, "neuro_exam_normal": True, "cost": 1200},
            "expected_decision": "Approved",
            "expected_outcome": "False Positive",
            "reason": "Should be denied: no LOC, vomiting, or abnormal exam; approved due to oversight"
        },
        {
            "id": "FN2",
            "domain": "Acute Care",
            "description": "Incorrect Denial of Necessary Labs",
            "scenario": "1-year-old female with fever (102.5°F) for 72 hours, lethargy, poor feeding; suspected sepsis",
            "service": "CBC and Blood Culture",
            "input": {"fever": 102.5, "duration_hours": 72, "systemic_signs": True, "cost": 50},
            "expected_decision": "Denied",
            "expected_outcome": "False Negative",
            "reason": "Should be approved: fever >101°F for >48h, systemic signs; denied due to glitch"
        }
    ],
    "Chronic Disease Management": [],
    "Preventive Care and Wellness": [],
    "Behavioral and Mental Health": [],
    "Neonatal and Perinatal Care": [],
    "Specialty Care": [],
    "Surgical and Procedural Care": [],
    "Pharmacy and Medication Management": [],
    "Rehabilitation and Therapy Services": [],
    "Diagnostic Services": []
}

In [26]:
# Cell 2: Define UR Agent and Test Harness
import logging
from typing import Dict, Any, List
import sys

# Set up logging with forced flush for Colab
logging.basicConfig(level=logging.INFO, format='%(message)s', force=True)
logger = logging.getLogger(__name__)

class URAgent:
    def __init__(self):
        self.policies = {
            "Abdominal CT scan": self._ct_scan_policy,
            "Chest X-Ray": self._chest_xray_policy,
            "CBC": self._cbc_policy,
            "Inpatient Admission": self._admission_policy,
            "Head CT scan": self._head_ct_policy,
            "CBC and Blood Culture": self._sepsis_lab_policy
        }

    def _ct_scan_policy(self, data: Dict[str, Any]) -> bool:
        return (data.get("pain_duration_hours", 0) > 12 or
                data.get("fever", 0) > 100.4 or
                data.get("tenderness", False))

    def _chest_xray_policy(self, data: Dict[str, Any]) -> bool:
        return (data.get("o2_sat", 100) < 95 or
                data.get("distress", False) or
                (data.get("fever", 0) > 101 and data.get("duration_days", 0) * 24 > 72))

    def _cbc_policy(self, data: Dict[str, Any]) -> bool:
        return (data.get("fever", 0) > 101 and data.get("duration_hours", 0) > 48) or data.get("systemic_signs", False)

    def _admission_policy(self, data: Dict[str, Any]) -> bool:
        if "o2_sat" in data:
            return (data.get("o2_sat", 100) < 95 or
                    data.get("distress", False) or
                    data.get("response_to_treatment", "") == "poor")
        if "urine_output_hours" in data:
            return (data.get("urine_output_hours", 0) > 8 and data.get("lethargy", False))
        return (data.get("fever", 0) > 102 and data.get("systemic_signs", True))

    def _head_ct_policy(self, data: Dict[str, Any]) -> bool:
        return (data.get("loc", False) or
                data.get("vomiting", False) or
                not data.get("neuro_exam_normal", True))

    def _sepsis_lab_policy(self, data: Dict[str, Any]) -> bool:
        return (data.get("fever", 0) > 101 and
                data.get("duration_hours", 0) > 48 and
                data.get("systemic_signs", False))

    def review_service(self, test_case: Dict[str, Any]) -> bool:
        service = test_case["service"]
        data = test_case["input"]
        policy = self.policies.get(service, lambda x: False)
        return policy(data)

class TestHarness:
    def __init__(self, agent: URAgent):
        self.agent = agent
        self.results = []

    def validate_result(self, test_case: Dict[str, Any], actual_decision: bool) -> bool:
        expected_decision = test_case["expected_decision"] == "Approved"
        expected_outcome = test_case["expected_outcome"]
        is_correct = actual_decision == expected_decision

        result_str = (f"Test {test_case['id']} - {test_case['description']} ({test_case['domain']}):\n"
                      f"  Expected: {test_case['expected_decision']} ({expected_outcome})\n"
                      f"  Reason: {test_case['reason']}\n"
                      f"  Actual: {'Approved' if actual_decision else 'Denied'}\n"
                      f"  Result: {'PASS' if is_correct else 'FAIL'}")
        self.results.append((test_case['id'], is_correct, result_str))

        return is_correct

    def run_tests(self, test_data: Dict[str, List[Dict[str, Any]]]):
        total_cost = 0
        total_savings = 0

        logger.info("=== Starting Test Execution ===")
        for domain, test_cases in test_data.items():
            if not test_cases:
                logger.info(f"No test cases for {domain}")
                continue

            logger.info(f"\n=== Running Tests for {domain} ===")
            for test_case in test_cases:
                actual_decision = self.agent.review_service(test_case)
                is_correct = self.validate_result(test_case, actual_decision)

                cost = test_case["input"]["cost"]
                bcbs_cost = cost * 0.3

                if actual_decision:
                    total_cost += bcbs_cost
                if test_case["expected_decision"] == "Approved" and not actual_decision:
                    total_savings -= bcbs_cost
                elif test_case["expected_decision"] == "Denied" and not actual_decision:
                    total_savings += bcbs_cost

                if not is_correct:
                    logger.warning(f"Mismatch in Test {test_case['id']}: Expected {test_case['expected_decision']}, Got {'Approved' if actual_decision else 'Denied'}")

        passed = sum(1 for _, is_correct, _ in self.results if is_correct)
        total_tests = len(self.results)
        failed = total_tests - passed
        logger.info(f"\n=== Overall Test Summary ===")
        logger.info(f"Total Tests: {total_tests}")
        logger.info(f"Passed: {passed}")
        logger.info(f"Failed: {failed}")
        logger.info(f"Total BCBS Cost: ${total_cost:,.2f}")
        logger.info(f"Total BCBS Savings: ${total_savings:,.2f}")
        logger.info("\n=== Detailed Results ===")
        for _, _, result in self.results:
            logger.info(result)

        # Force flush to ensure output in Colab
        sys.stdout.flush()

In [27]:
# Cell 3: Execute the simulation without reprinting test cases
harness = TestHarness(agent=URAgent())
harness.run_tests(test_cases)

=== Starting Test Execution ===

=== Running Tests for Acute Care ===
Mismatch in Test FP1: Expected Approved, Got Denied
Mismatch in Test FN1: Expected Denied, Got Approved
Mismatch in Test FP2: Expected Approved, Got Denied
Mismatch in Test FN2: Expected Denied, Got Approved
No test cases for Chronic Disease Management
No test cases for Preventive Care and Wellness
No test cases for Behavioral and Mental Health
No test cases for Neonatal and Perinatal Care
No test cases for Specialty Care
No test cases for Surgical and Procedural Care
No test cases for Pharmacy and Medication Management
No test cases for Rehabilitation and Therapy Services
No test cases for Diagnostic Services

=== Overall Test Summary ===
Total Tests: 8
Passed: 4
Failed: 4
Total BCBS Cost: $9,375.00
Total BCBS Savings: $1,176.00

=== Detailed Results ===
Test TP1 - Approval of Necessary CT Scan (Acute Care):
  Expected: Approved (True Positive)
  Reason: Meets AAP criteria for suspected appendicitis: pain >12h, feve